# Cleaning Crime Data
#### By: Jack Cohen, Karina Hutula, Raheem Paxton

# Import Dependencies

In [180]:
import pandas as pd
import pandas_gbq
import os
import datetime as datetime
import pytz
import time
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
t = time.time()

# Access Credentials and Import Data

In [15]:
# Access Google Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="apikey.json"

# Save crime data as crime_df DataFrame
crime_df = pandas_gbq.read_gbq(query="""SELECT * FROM `bigquery-public-data.chicago_crime.crime`""", use_bqstorage_api=True)

In [111]:
# Read columns available
print('Raw Data Columns')
print('---------------------')
columns = crime_df.columns
for x in columns:
    print(x)
    
# Preview Data
crime_df.head()

Raw Data Columns
---------------------
unique_key
case_number
date
block
iucr
primary_type
description
location_description
arrest
domestic
beat
district
ward
community_area
fbi_code
x_coordinate
y_coordinate
year
updated_on
latitude
longitude
location


,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,2611725,HJ212994,2003-03-01 21:45:42+00:00,081XX S MARYLAND AVE,4387,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,APARTMENT,True,True,...,8.0,44.0,26,1183316.0,1851309.0,2003,2018-02-28 15:56:25+00:00,41.747185,-87.603851,"(41.747184808, -87.603851467)"
1,8072156,HT304014,2011-05-14 13:24:00+00:00,081XX S DR MARTIN LUTHER KING JR DR,4387,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,APARTMENT,True,False,...,6.0,44.0,26,1180319.0,1851022.0,2011,2018-02-10 15:50:01+00:00,41.746466,-87.614842,"(41.746466445, -87.614841984)"
2,2580366,HJ170817,2003-02-07 16:00:00+00:00,091XX S COTTAGE GROVE AVE,4387,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,BARBERSHOP,False,True,...,8.0,44.0,26,1183157.0,1844715.0,2003,2018-02-10 15:50:01+00:00,41.729094,-87.604639,"(41.729093835, -87.604638666)"
3,8124600,HT359572,2011-06-21 22:39:00+00:00,079XX S DOBSON AVE,4387,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,APARTMENT,False,True,...,8.0,44.0,26,1184602.0,1852543.0,2011,2018-02-10 15:50:01+00:00,41.750541,-87.599101,"(41.750541016, -87.599100677)"
4,12162170,JD363106,2020-09-10 17:38:00+00:00,008XX E 82ND ST,4387,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,RESIDENCE,False,True,...,8.0,44.0,26,1183457.0,1850811.0,2020,2020-09-17 15:47:29+00:00,41.745815,-87.603350,"(41.745814961, -87.60335029)"


In [112]:
# Use test_df to test all transformations to save proccessing and time
test_df = crime_df.head(100)

# Drop Unnecessary Columns

In [113]:
# Drop columns with irrelevant data
df2=crime_df.drop(columns=['unique_key',
                          'case_number',
                          'block',
                          'iucr',
                          'beat',
                          'district',
                          'ward',
                          'community_area',
                          'fbi_code',
                          'x_coordinate',
                          'y_coordinate',
                          'year',
                          'updated_on',
                          'latitude',
                          'longitude',
                          'location'])

df2.tail()

,date,primary_type,description,location_description,arrest,domestic
7393739,2006-05-17 07:35:00+00:00,SEX OFFENSE,PUBLIC INDECENCY,VEHICLE NON-COMMERCIAL,True,False
7393740,2006-08-12 02:56:00+00:00,BATTERY,AGG PO HANDS NO/MIN INJURY,GAS STATION,True,False
7393741,2006-10-29 06:25:00+00:00,ASSAULT,AGGRAVATED: OTHER DANG WEAPON,STREET,False,False
7393742,2009-12-24 07:07:00+00:00,OTHER OFFENSE,FALSE/STOLEN/ALTERED TRP,STREET,False,False
7393743,2013-09-10 14:06:00+00:00,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,SIDEWALK,True,False


# Timezone Adjustment

In [114]:
# Adjust dt for timezone and round time to nearest hour (to match weather data)

timezone = pytz.timezone("America/Chicago")
local_datetime = []
rounded_datetime = []

for x in df2['date']:
    x = x.to_pydatetime()
    utc_dt = x.tzinfo.fromutc(x)
    local_dt = utc_dt.astimezone(timezone)
    time = pd.Timestamp(local_dt)
    try:
        rounded = time.round(freq='H')
    except:
        print(f"Error at : {local_dt}")
        rounded = local_dt
    
    rounded_string = rounded.strftime("%Y-%m-%d %H:%M")
    local_string = local_dt.strftime("%Y-%m-%d %H:%M")
    
    rounded_datetime.append(rounded_string)
    local_datetime.append(local_string)

# Add adjusted time to DataFrame

df2.insert(0,'exact_dt',local_datetime)
df2.insert(0,'local_dt',rounded_datetime)

Error at : 2016-11-06 01:00:00-05:00
Error at : 2006-10-29 01:29:48-06:00
Error at : 2021-03-14 01:48:00-06:00
Error at : 2015-11-01 01:00:00-06:00
Error at : 2009-11-01 01:18:27-06:00
Error at : 2004-10-31 01:10:00-05:00
Error at : 2015-03-08 01:30:00-06:00
Error at : 2011-11-06 01:15:00-06:00
Error at : 2004-10-31 01:00:00-06:00
Error at : 2002-10-27 01:00:00-06:00
Error at : 2007-03-11 01:46:57-06:00
Error at : 2005-04-03 01:35:00-06:00
Error at : 2005-04-03 01:30:00-06:00
Error at : 2013-11-03 01:00:00-06:00
Error at : 2020-11-01 01:20:00-06:00
Error at : 2015-11-01 01:00:00-06:00
Error at : 2003-10-26 01:00:00-05:00
Error at : 2008-11-02 01:00:00-06:00
Error at : 2018-11-04 00:39:00-05:00
Error at : 2014-03-09 01:31:00-06:00
Error at : 2006-04-02 01:30:00-06:00
Error at : 2016-03-13 01:43:00-06:00
Error at : 2009-11-01 01:00:00-05:00
Error at : 2007-03-11 01:45:00-06:00
Error at : 2010-11-07 01:15:00-06:00
Error at : 2020-11-01 01:15:00-05:00
Error at : 2016-11-06 01:00:00-05:00
E

Error at : 2020-11-01 01:00:00-06:00
Error at : 2014-11-02 01:15:00-06:00
Error at : 2014-11-02 00:34:00-05:00
Error at : 2001-10-28 01:29:01-05:00
Error at : 2001-10-28 01:00:00-06:00
Error at : 2001-04-01 01:45:00-06:00
Error at : 2001-10-28 01:00:00-05:00
Error at : 2002-04-07 01:40:00-06:00
Error at : 2014-11-02 01:25:00-06:00
Error at : 2019-11-03 01:00:00-05:00
Error at : 2012-03-11 01:50:00-06:00
Error at : 2017-11-05 01:00:00-05:00
Error at : 2009-11-01 01:00:00-06:00
Error at : 2004-10-31 01:00:00-06:00
Error at : 2017-11-05 01:14:00-05:00
Error at : 2018-03-11 01:45:00-06:00
Error at : 2018-11-04 01:25:00-05:00
Error at : 2007-11-04 01:00:00-06:00
Error at : 2008-11-02 01:00:00-06:00
Error at : 2006-04-02 01:55:00-06:00
Error at : 2008-11-02 01:05:00-06:00
Error at : 2015-11-01 01:00:00-05:00
Error at : 2006-10-29 01:14:22-05:00
Error at : 2008-11-02 01:00:00-05:00
Error at : 2005-10-30 01:25:00-06:00
Error at : 2006-10-29 01:00:00-06:00
Error at : 2003-10-26 01:00:00-06:00
E

Error at : 2020-11-01 01:00:00-06:00
Error at : 2018-11-04 01:21:00-06:00
Error at : 2008-11-02 01:00:00-05:00
Error at : 2009-11-01 01:00:00-06:00
Error at : 2020-11-01 00:39:00-05:00
Error at : 2006-10-29 01:20:00-06:00
Error at : 2003-10-26 01:00:00-05:00
Error at : 2014-11-02 01:00:00-06:00
Error at : 2020-11-01 01:20:00-06:00
Error at : 2005-10-30 01:05:00-06:00
Error at : 2019-03-10 01:30:00-06:00
Error at : 2021-03-14 01:57:00-06:00
Error at : 2010-03-14 01:38:00-06:00
Error at : 2002-10-27 01:00:00-05:00
Error at : 2019-11-03 01:12:00-05:00
Error at : 2018-03-11 01:55:00-06:00
Error at : 2020-03-08 01:45:00-06:00
Error at : 2004-10-31 01:15:00-05:00
Error at : 2014-03-09 01:45:00-06:00
Error at : 2015-11-01 01:16:00-06:00
Error at : 2011-03-13 01:30:00-06:00
Error at : 2010-03-14 01:55:00-06:00
Error at : 2007-11-04 01:00:00-05:00
Error at : 2015-11-01 01:20:00-06:00
Error at : 2016-03-13 01:30:00-06:00
Error at : 2002-10-27 01:08:07-05:00
Error at : 2017-03-12 01:38:00-06:00
E

# Join DataFrames

In [ ]:
# Import Weather DataFrame from other script; Rename Crime DataFrame

crime_df = df2.drop(columns=['date'])

In [183]:
weather_df = pd.read_csv('Resources/weather_df.csv')

In [184]:
# Join Weather and Crime DataFrames

left = crime_df
right = weather_df
crime_weather_df = pd.merge(left, right, on="local_dt", how="left")

In [185]:
crime_weather_df.tail(50)

,local_dt,exact_dt,primary_type,description,location_description,arrest,domestic,temp_F,feels_like_F,temp_min_F,...,pressure_hPa,humidity_percent,wind_speed_mph,wind_deg,rain_1h_inches,snow_1h_inches,clouds_percent,weather_id,weather_main,weather_description
7393694,2010-02-27 11:00,2010-02-27 11:00,ASSAULT,PRO EMP HANDS NO/MIN INJURY,GOVERNMENT BUILDING/PROPERTY,True,False,31.48,21.34,29.73,...,1012.0,74.0,13.87,330.0,0.00,0.00,90.0,[804],['Clouds'],['overcast clouds']
7393695,2011-09-08 10:00,2011-09-08 10:14,NARCOTICS,POSS: CANNABIS MORE THAN 30GMS,RESIDENCE,True,False,65.77,64.81,63.39,...,1018.0,59.0,16.11,20.0,0.00,0.00,40.0,[802],['Clouds'],['scattered clouds']
7393696,2012-09-10 09:00,2012-09-10 08:35,ASSAULT,AGGRAVATED:KNIFE/CUTTING INSTR,STREET,False,False,65.25,64.15,59.05,...,1022.0,57.0,1.14,141.0,0.00,0.00,40.0,[802],['Clouds'],['scattered clouds']
7393697,2015-11-18 08:00,2015-11-18 08:00,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,"SCHOOL, PUBLIC, BUILDING",False,False,59.07,58.06,57.22,...,1000.0,72.0,23.04,160.0,0.01,0.00,90.0,[500],['Rain'],['light rain']
7393698,2019-04-08 03:00,2019-04-08 02:56,DECEPTIVE PRACTICE,COUNTERFEITING DOCUMENT,CURRENCY EXCHANGE,False,False,56.84,56.26,54.90,...,1011.0,86.0,3.36,260.0,0.00,0.00,75.0,[803],['Clouds'],['broken clouds']
7393699,2002-07-01 06:00,2002-07-01 06:00,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,OTHER,False,False,78.44,79.74,76.15,...,1019.0,80.0,8.05,250.0,0.00,0.00,1.0,[800],['Clear'],['sky is clear']
7393700,2006-02-14 06:00,2006-02-14 05:54,THEFT,ATTEMPT THEFT,DRUG STORE,False,False,26.31,18.70,25.21,...,1011.0,79.0,6.93,240.0,0.00,0.00,20.0,[801],['Clouds'],['few clouds']
7393701,2006-04-16 17:00,2006-04-16 17:00,MOTOR VEHICLE THEFT,"TRUCK, BUS, MOTOR HOME",STREET,False,False,53.92,52.77,50.34,...,1001.0,80.0,27.74,100.0,0.16,0.00,90.0,[502],['Rain'],['heavy intensity rain']
7393702,2006-10-11 04:00,2006-10-11 04:30,OBSCENITY,OBSCENE MATTER,RESIDENCE,True,False,57.76,57.60,53.29,...,1001.0,93.0,8.05,220.0,0.05,0.00,90.0,[501],['Rain'],['moderate rain']
7393703,2008-07-17 15:00,2008-07-17 14:40,DECEPTIVE PRACTICE,COUNTERFEITING DOCUMENT,RESTAURANT,False,False,87.89,88.59,85.53,...,1018.0,43.0,10.29,210.0,0.00,0.00,75.0,[803],['Clouds'],['broken clouds']


# Show Available Data

In [186]:
# Column Headers
print('AVAILABLE INFORMATION')
print('------------------------')
columns = crime_weather_df.columns
for x in columns:
    print(x)

AVAILABLE INFORMATION
------------------------
local_dt
exact_dt
primary_type
description
location_description
arrest
domestic
temp_F
feels_like_F
temp_min_F
temp_max_F
pressure_hPa
humidity_percent
wind_speed_mph
wind_deg
rain_1h_inches
snow_1h_inches
clouds_percent
weather_id
weather_main
weather_description


In [187]:
# Show frequency of crime types in dataset
print('FREQUENCY OF CRIME TYPE')
print('------------------------')
print(crime_weather_df['primary_type'].value_counts())

FREQUENCY OF CRIME TYPE
------------------------
THEFT                                1555663
BATTERY                              1355941
CRIMINAL DAMAGE                       842931
NARCOTICS                             738952
ASSAULT                               473110
OTHER OFFENSE                         459726
BURGLARY                              411740
MOTOR VEHICLE THEFT                   340742
DECEPTIVE PRACTICE                    316904
ROBBERY                               277428
CRIMINAL TRESPASS                     207380
WEAPONS VIOLATION                      92254
PROSTITUTION                           69422
OFFENSE INVOLVING CHILDREN             51954
PUBLIC PEACE VIOLATION                 51152
SEX OFFENSE                            28514
CRIM SEXUAL ASSAULT                    27853
INTERFERENCE WITH PUBLIC OFFICER       17708
GAMBLING                               14602
LIQUOR LAW VIOLATION                   14546
ARSON                                  12513
HOMICI

# Export CSV

In [188]:
# Export to CSV in Resources folder to then upload to PostgreSQL
crime_weather_df.to_csv('Resources/crime_weather_df.csv',index=False)

In [ ]:
# Alert completion
import os
os.system('say "Your program has finished."')
elapsed = (time.time()-t)/60
print(f"Code ran in {elapsed} minutes")

In [200]:
crime_weather_df.head()

,local_dt,exact_dt,primary_type,description,location_description,arrest,domestic,temp_F,feels_like_F,temp_min_F,...,pressure_hPa,humidity_percent,wind_speed_mph,wind_deg,rain_1h_inches,snow_1h_inches,clouds_percent,weather_id,weather_main,weather_description
0,2003-03-01 16:00,2003-03-01 15:45,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,APARTMENT,True,True,33.64,25.61,33.17,...,1013.0,64.0,10.29,230.0,0.00,0.0,90.0,[804],['Clouds'],['overcast clouds']
1,2011-05-14 08:00,2011-05-14 08:24,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,APARTMENT,True,False,46.81,39.25,46.11,...,1006.0,92.0,20.80,10.0,0.00,0.0,90.0,"[300, 701]","['Drizzle', 'Mist']","['light intensity drizzle', 'mist']"
2,2003-02-07 10:00,2003-02-07 10:00,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,BARBERSHOP,False,True,15.06,2.46,12.25,...,1024.0,66.0,11.41,290.0,0.00,0.0,1.0,[800],['Clear'],['sky is clear']
3,2011-06-21 18:00,2011-06-21 17:39,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,APARTMENT,False,True,80.89,83.05,78.44,...,1002.0,61.0,13.87,170.0,0.04,0.0,90.0,[500],['Rain'],['light rain']
4,2020-09-10 13:00,2020-09-10 12:38,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,RESIDENCE,False,True,65.23,64.74,62.49,...,1028.0,70.0,14.99,30.0,0.00,0.0,90.0,[804],['Clouds'],['overcast clouds']


# Add Table to DataBase Using SQLAlchemy

In [258]:
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Boolean, delete

In [259]:
# Create CW Class

class CW(Base):
    __tablename__ = 'crime_weather'
    id = Column(Integer, primary_key=True)
    local_dt = Column(String(255))
    exact_dt = Column(String(255))
    primary_type = Column(String(255))
    description = Column(String(255))
    location_description = Column(String(255))
    arrest = Column(Boolean)
    domestic = Column(Boolean)
    temp_F = Column(Float)
    feels_like_F = Column(Float)
    temp_min_F = Column(Float)
    temp_max_F = Column(Float)
    pressure_hPa = Column(Float)
    humidity_percent = Column(Float)
    wind_speed_mph = Column(Float)
    wind_deg = Column(Float)
    rain_1h_inches = Column(Float)
    snow_1h_inches = Column(Float)
    clouds_percent = Column(Float)
    weather_id = Column(String(255))
    weather_main = Column(String(255))
    weather_description = Column(String(255))


In [260]:
# Create Database Connection
# ----------------------------------
# Establish Connection
engine = create_engine("sqlite:///crime_weather.sqlite")
conn = engine.connect()

In [261]:
Base.metadata.create_all(conn)

In [262]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [264]:
for x in range(len(crime_weather_df)):
    info = CW(id = x+1,
            local_dt = crime_weather_df['local_dt'][x],
            exact_dt = crime_weather_df['exact_dt'][x],
            primary_type = crime_weather_df['primary_type'][x],
            description = crime_weather_df['description'][x],
            location_description = crime_weather_df['location_description'][x],
            arrest = crime_weather_df['arrest'][x],
            domestic = crime_weather_df['domestic'][x],
            temp_F = crime_weather_df['temp_F'][x],
            feels_like_F = crime_weather_df['feels_like_F'][x],
            temp_min_F = crime_weather_df['temp_min_F'][x],
            temp_max_F = crime_weather_df['temp_max_F'][x],
            pressure_hPa = crime_weather_df['pressure_hPa'][x],
            humidity_percent = crime_weather_df['humidity_percent'][x],
            wind_speed_mph = crime_weather_df['wind_speed_mph'][x],
            wind_deg = crime_weather_df['wind_deg'][x],
            rain_1h_inches = crime_weather_df['rain_1h_inches'][x],
            snow_1h_inches = crime_weather_df['snow_1h_inches'][x],
            clouds_percent = crime_weather_df['clouds_percent'][x],
            weather_id = crime_weather_df['weather_id'][x],
            weather_main = crime_weather_df['weather_main'][x],
            weather_description = crime_weather_df['weather_description'][x],
            )
    session.add(info)
    session.commit()


In [265]:
import os
os.system('say "Your program has finished."')

0